<a href="https://colab.research.google.com/github/componavt/LLLE-R1900s/blob/main/src/visualization/radial_loan_stars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌐 Radial Loan Profiles: Bilingual Radar Charts per Settlement

Each village is a star — rays = credit categories.  
**Ray length** = total money borrowed.  
**Label at tip** = number of loans.

✨ **Features:**
- 🌍 Choose language: Russian (`ru`) or English (`en`)
- 🏘️ Settlement name: `Russian` or `English` from `society_settlement.csv`
- 📏 Axis labels: `loan_short_ru` (RU) or `Name` (EN) from `credit_items.csv`
- 🔢 Loan count shown as number
- 🖼️ Rendered directly in Colab

⚙️ Configurable via `USE_LANGUAGE` variable  
✅ Fully bilingual visualization

⚙️ [2] Install Dependencies & Set Language

In [1]:
# Install compatible versions for static image rendering in Colab
#!pip install -q python-dotenv pandas plotly>=6.1.1 kaleido
!pip install -q python-dotenv pandas plotly

# Clone the repo if running in Colab (optional — if data not uploaded manually)
import os

if not os.path.exists('LLLE-R1900s'):
    !git clone https://github.com/componavt/LLLE-R1900s.git
    %cd LLLE-R1900s
else:
    %cd LLLE-R1900s

# === LANGUAGE SWITCH ===
USE_LANGUAGE = "ru"
assert USE_LANGUAGE in ("ru", "en"), "USE_LANGUAGE must be 'ru' or 'en'"

Cloning into 'LLLE-R1900s'...
remote: Enumerating objects: 511, done.
remote: Counting objects: 100% (511/511), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 511 (delta 383), reused 394 (delta 294), pack-reused 0 (from 0)
Receiving objects: 100% (511/511), 1.36 MiB | 4.69 MiB/s, done.
Resolving deltas: 100% (383/383), done.
/content/LLLE-R1900s


📥 [3] Load & Prepare Data with Quadrant Mapping

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv

# Load configuration
load_dotenv('config.env')

# Paths
csv_out_dir = os.getenv('CSV_OUT_DIR', 'data/csv_out')
output_file_name = os.getenv('OUTPUT_CSV_FILE')

if not output_file_name:
    csv_files = [f for f in os.listdir(csv_out_dir) if f.endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError("No CSV files found in the output directory.")
    output_file_name = csv_files[0]

csv_path = os.path.join(csv_out_dir, output_file_name)
print(f"Loading loan data from: {csv_path}")

# Load main loan data
df_loans = pd.read_csv(csv_path)
print(f"Loaded {len(df_loans)} loan records.")

# Load credit items (for axis labels + quadrant logic)
df_credit = pd.read_csv('data/credit_items.csv')
print(f"Loaded {len(df_credit)} credit item definitions.")

# Load settlement metadata
df_society = pd.read_csv('data/society_settlement.csv')
print(f"Loaded {len(df_society)} settlements.")

# --- Build settlement display name map ---
if USE_LANGUAGE == "ru":
    df_society['display_settlement'] = df_society['Russian']
else:
    df_society['display_settlement'] = df_society['English']

settlement_name_map = dict(zip(df_society['English'], df_society['display_settlement']))

# Validate settlements
unknown_settlements = set(df_loans['settlement']) - set(df_society['English'])
if unknown_settlements:
    print("⚠️ Warning: Unknown settlements in loan data:", unknown_settlements)

# --- Build credit item display label map ---
if USE_LANGUAGE == "ru":
    df_credit['display_label'] = df_credit['loan_short_ru']
else:
    df_credit['display_label'] = df_credit['Name']

label_map = dict(zip(df_credit['Name'], df_credit['display_label']))

# Validate credit items
unknown_items = set(df_loans['credit_item']) - set(df_credit['Name'])
if unknown_items:
    print("⚠️ Warning: Unknown credit_item values:", unknown_items)

# Add display settlement name to loan data
df_loans['display_settlement'] = df_loans['settlement'].map(settlement_name_map)
df_loans = df_loans.dropna(subset=['display_settlement'])

# --- Map credit items to quadrants ---
def assign_quadrant(row):
    lt = row['loan_type']
    aci = row['aci_status']

    is_prod = lt in ('productive', 'conditionally_productive')
    is_aci = aci in ('aci', 'conditionally_aci')

    if is_prod and is_aci:
        return 1
    elif not is_prod and is_aci:
        return 2
    elif not is_prod and not is_aci:
        return 3
    elif is_prod and not is_aci:
        return 4
    else:
        return 0  # fallback

df_credit['quadrant'] = df_credit.apply(assign_quadrant, axis=1)

# Create ordered list of credit items by quadrant → within-quadrant alphabetical
df_credit_sorted = df_credit.sort_values(['quadrant', 'Name'])
all_credit_items = df_credit_sorted['Name'].tolist()
axis_labels = [label_map[item] for item in all_credit_items]

print("✅ Data prepared with quadrant-aware credit item ordering.")

Loading loan data from: data/csv_out/loans_s28_i21.csv
Loaded 1768 loan records.
Loaded 21 credit item definitions.
Loaded 29 settlements.
✅ Data prepared with quadrant-aware credit item ordering.


🌟 [4] Individual Settlement Radar Charts (Quadrant Layout)

In [3]:
import plotly.graph_objects as go

# Reuse sorted credit items from previous cell
# axis_labels and all_credit_items already defined

# Aggregate loan data
df_agg = df_loans.groupby(['display_settlement', 'credit_item']).agg(
    total_amount=('amount_rubles', 'sum'),
    total_count=('loan_count', 'sum')
).reset_index()

# Unique display settlement names
settlements = df_agg['display_settlement'].unique()
shown_count = 0

for settlement in settlements:
    df_set = df_agg[df_agg['display_settlement'] == settlement]

    # Build full vectors (including zeros)
    amounts = []
    counts = []
    for item in all_credit_items:
        match = df_set[df_set['credit_item'] == item]
        if not match.empty:
            amt = match['total_amount'].iloc[0]
            cnt = int(match['total_count'].iloc[0])
        else:
            amt = 0
            cnt = 0
        amounts.append(amt)
        counts.append(cnt)

    if sum(amounts) == 0:
        continue

    # Close the loop
    theta = axis_labels + [axis_labels[0]]
    r = amounts + [amounts[0]]
    text_labels = [str(c) if c > 0 else '' for c in counts] + ['']

    # Create figure
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=r,
        theta=theta,
        fill='toself',
        line_color='steelblue',
        opacity=0.6,
        showlegend=False
    ))

    fig.add_trace(go.Scatterpolar(
        r=r,
        theta=theta,
        mode='markers+text',
        text=text_labels,
        textposition='top center',
        marker=dict(size=8, color='darkred'),
        showlegend=False
    ))

    title = f"Loan Profile: {settlement}" if USE_LANGUAGE == "en" else f"Профиль ссуд: {settlement}"
    radial_title = "Total Amount (Rubles)" if USE_LANGUAGE == "en" else "Сумма ссуд (руб.)"

    fig.update_layout(
        title=title,
        polar=dict(
            radialaxis=dict(visible=True, title=radial_title),
            angularaxis=dict(direction="clockwise")
        ),
        showlegend=False,
        width=700,
        height=700,
        font=dict(size=10)
    )

    fig.show(renderer="colab")
    shown_count += 1

print(f"\n🎉 Displayed {shown_count} quadrant-organized radar charts in {'Russian' if USE_LANGUAGE == 'ru' else 'English'}.")


🎉 Displayed 28 quadrant-organized radar charts in Russian.


📅 [5] Time-Stacked Radar Chart with Quadrant Labels (All Settlements, by Year)

📓 Ячейка 5: Общая радиальная диаграмма по всем поселениям

In [6]:
import plotly.graph_objects as go
import plotly.colors as pc
import numpy as np

# Reuse sorted credit items and quadrant info
df_credit_sorted = df_credit.sort_values(['quadrant', 'Name'])
all_credit_items = df_credit_sorted['Name'].tolist()
axis_labels = [label_map[item] for item in all_credit_items]
item_to_quadrant = dict(zip(df_credit['Name'], df_credit['quadrant']))
quadrants = [item_to_quadrant[item] for item in all_credit_items]

# Aggregate yearly data
df_yearly = df_loans.groupby(['year', 'credit_item']).agg(
    total_amount=('amount_rubles', 'sum')
).reset_index()
years = sorted(df_yearly['year'].unique())

# Build per-year amounts (not cumulative!)
yearly_amounts = {}
for year in years:
    yearly_amounts[year] = []
    for item in all_credit_items:
        amt = df_yearly[(df_yearly['year'] == year) & (df_yearly['credit_item'] == item)]['total_amount'].sum()
        yearly_amounts[year].append(amt)

# Prepare theta (closed loop)
theta_full = axis_labels + [axis_labels[0]]

# === Build stacked rings manually ===
fig = go.Figure()

# Start from zero
r_prev = [0] * len(all_credit_items)

# Color palette for years
color_seq = pc.qualitative.Bold
if len(years) > len(color_seq):
    color_seq = (color_seq * ((len(years) // len(color_seq)) + 1))[:len(years)]

for i, year in enumerate(years):
    r_curr = [r_prev[j] + yearly_amounts[year][j] for j in range(len(all_credit_items))]

    # Create ring: go from r_curr forward, then r_prev backward
    r_ring = r_curr + r_prev[::-1] + [r_curr[0]]  # close loop
    theta_ring = theta_full + axis_labels[::-1] + [theta_full[0]]

    fig.add_trace(go.Scatterpolar(
        r=r_ring,
        theta=theta_ring,
        mode='lines',
        fill='toself',
        fillcolor=color_seq[i],
        line=dict(width=0),
        opacity=0.7,
        name=str(year),
        showlegend=True
    ))

    r_prev = r_curr  # update for next year

# === Add quadrant labels as text on polar plot ===
# Find segment boundaries
segment_starts = [0]
current_quad = quadrants[0]
for i in range(1, len(quadrants)):
    if quadrants[i] != current_quad:
        segment_starts.append(i)
        current_quad = quadrants[i]
segment_starts.append(len(quadrants))

# Quadrant labels
if USE_LANGUAGE == "ru":
    QUADRANT_LABELS = {
        1: "производственная, ркп",
        2: "непроизводственная, ркп",
        3: "непроизводственная, не ркп",
        4: "производственная, не ркп"
    }
else:
    QUADRANT_LABELS = {
        1: "Productive, ACI",
        2: "Non-productive, ACI",
        3: "Non-productive, Non-ACI",
        4: "Productive, Non-ACI"
    }

# Place one label per quadrant at mid-angle
label_thetas = []
label_rs = []
label_texts = []

max_total = max(r_prev) if r_prev else 100
label_radius = max_total * 1.3 if max_total > 0 else 100

for k in range(len(segment_starts) - 1):
    start = segment_starts[k]
    end = segment_starts[k + 1]
    quad = quadrants[start]

    # Mid index
    mid_idx = (start + end - 1) // 2
    mid_theta = axis_labels[mid_idx]
    label_thetas.append(mid_theta)
    label_rs.append(label_radius)
    label_texts.append(QUADRANT_LABELS[quad])

# Add text labels
fig.add_trace(go.Scatterpolar(
    r=label_rs,
    theta=label_thetas,
    mode='text',
    text=label_texts,
    textfont=dict(size=12, color='black'),
    showlegend=False
))

# === Final layout ===
title = "Сводный профиль ссуд по годам и квадрантам" if USE_LANGUAGE == "ru" else "Combined Loan Profile by Year & Quadrants"
radial_title = "Сумма ссуд (руб.)" if USE_LANGUAGE == "ru" else "Loan Amount (Rubles)"

fig.update_layout(
    title=title,
    polar=dict(
        radialaxis=dict(visible=True, title=radial_title),
        angularaxis=dict(direction="clockwise", tickfont=dict(size=9))
    ),
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.02,
        font=dict(size=11)
    ),
    width=850,
    height=800,
    font=dict(size=10)
)

fig.show(renderer="colab")
print(f"✅ Time-stacked (ring-based) radar chart with quadrant labels for {len(years)} years.")

✅ Time-stacked (ring-based) radar chart with quadrant labels for 7 years.
